<a href="https://colab.research.google.com/github/Abudhagir/Abudhagir/blob/main/Dr_Syed_EVA8_Pytorch_Ex_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn  # consiss all basic blocks such as conv layers, activaion fns, polling, padding layers ec.
import torch.nn.functional as F   #consists of functions of Conv, poolin, padding, activation etc..
import torch.optim as optim     # package for implemtaion of opimization algo
from torchvision import datasets, transforms   # Utility class for buliding own datasets, transforms for Image transformation

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input -1*28*28,  OUtput-32*28*28, kernal- 3*3,  RF -3*3  
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  #input -32*28*28, OUtput-64*28*28, kernal- 3*3,  RF - 5*5 
        self.pool1 = nn.MaxPool2d(2, 2)               #input - 64*28*28 Pooling - 2 * 2 ,  output channel - 64*14*14, RF - 10*10 
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # input -64*14*14, OUtput-128*14*14, kernal- 3*3, RF -12*12 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)# input -128 *14*14,OUtput-256*14*14, kernal- 3*3, RF - 14*14
        self.pool2 = nn.MaxPool2d(2, 2)               # input- 256*14*14. Pooling - 2 * 2,  output channel - 256*7*7, RF -28*28
        self.conv5 = nn.Conv2d(256, 512, 3)           # input -256*7*7,OUtput-512*5*5, kernal- 3*3, RF -30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)          # input -512*5*5,OUtput-1024*3*3, kernal- 3*3, RF-32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)           # input -1024*3*3,OUtput-10*1*1, kernal- 3*3, RF - 34*34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   # Relu activation fn at Layer 1 & 2
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))   # Relu activation fn at Layer 3 & 4 
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               # Relu activation fn at Layer 5 & 6
        #x = F.relu(self.conv7(x))                                   # Relu activation fn at Layer 7
        
                    # Never apply relu acivation funtion in last layer, model can't learn negative values, 
                    # Due to this reason loss won't decrease, accuracy also low. 
        x = self.conv7(x)                                           # performs conv operation at layer 7

        x = x.view(-1, 10)                                          # reshape the tensor array                             
        return F.log_softmax(x)                                    # Softmax activation fn at last layer for 10 classes

In [13]:
!pip install torchsummary
from torchsummary import summary                       # visualization of the model
use_cuda = torch.cuda.is_available()                   # To check whether GPU is available (CUDA enabled)
device = torch.device("cuda" if use_cuda else "cpu")   # If CUDA is there Use GPU else use CPU
model = Net().to(device)                               # copy the data to GPU
summary(model, input_size=(1, 28, 28))                 # Give abt the model summary for the input shape of (1, 28 * 28)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 

<ipython-input-12-7165e5d9bb9a>:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)                                    # Softmax activation fn at last layer- 10 classes


In [7]:


torch.manual_seed(1)   #Sets the seed for generating random numbers
batch_size = 128       # no of training samples used in one iteration 

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  # 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                       # data loading load the MNIST dataset  - for Training        
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                      # data split for Test


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [8]:
from tqdm import tqdm                     # to see the Progress of the model training and testing
def train(model, device, train_loader, optimizer, epoch):   # passing the model which created before, device-GPU or CPU, training samples, opimizer and no of iteraion to train the model
    model.train()
    pbar = tqdm(train_loader)     # wrapping the iteratable training sample in with tqdm
    for batch_idx, (data, target) in enumerate(pbar):     # Iterate tqdm 
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()                           # clears x.grad for every parameter x in the optimizer
        output = model(data)
        loss = F.nll_loss(output, target)               #  negative log likelihood loss - classification prob
        loss.backward()                                 # compute back ptopagation- dl/dx
        optimizer.step()                                # perform single optimization step
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  #Progress bar description


def test(model, device, test_loader):
    model.eval()                            # ses the module in evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():                   # Disabling gradient calculation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()        # 

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   # stochastic Gradient Descent optimizer is used, learning rate - 0.01, momentum-0.9

for epoch in range(1, 2):                                 # no of epoch - 2
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-12-7165e5d9bb9a>:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)                                    # Softmax activation fn at last layer- 10 classes
loss=0.08588224649429321 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.66it/s]



Test set: Average loss: 0.0737, Accuracy: 9753/10000 (98%)

